In [1]:
import json
%run lib.ipynb

In [2]:
final_games = [
    "27ef7b5a-c150-4f7b-9584-7d003375f161",
    "bf7da94c-5ea2-4467-b940-240c101d487a",
    "ad473e94-3654-447d-a5ba-52de87eb0976",
    "ab9c7558-cb00-488d-829e-83ed7c228a4c",
    "18d4add9-6e6b-45ed-88bd-84198cbc84ca",
    "bc8b6ed3-35b9-49b6-a08f-bcf5d29d5c26",
    "5c39dd5c-1953-42b9-9cf1-47001d62c7bf"
]

In [3]:
report = {
    'Panthers': {},
    'Oilers': {}
}

In [4]:



def parseGame(gameData):
    for _period in gameData.periods:
        for _event in _period.events:
    
            team = ''
            player = None
            action_area = ''
            eventReportObj = None
            
            if not hasattr(_event, 'attribution'):
                continue
            
            team = _event.attribution.name
                           
            if team != 'Oilers' and team != 'Panthers':
                continue
                
            if hasattr(_event,'location'):
                 if hasattr(_event.location,'action_area'):
                    action_area = _event.location.action_area 
            
            # STEP TYPE: hits
            if _event.event_type == "hit":
                for _statistic in _event.statistics:
                    if _statistic.type == 'hit':
                            player = _statistic.player.full_name

                eventReportObj = {
                    'type': _event.event_type,
                    'action_area' :  action_area ,
                    'location' : [_event.location.coord_x, _event.location.coord_y ]
                }
                
            # STEP TYPE: takeaway
            if _event.event_type == "takeaway":
                for _statistic in _event.statistics:
                    if _statistic.type == 'takeaway':
                            player = _statistic.player.full_name

                eventReportObj = {
                    'type': _event.event_type,
                    'action_area' :  action_area ,
                    'location' : [_event.location.coord_x, _event.location.coord_y ]
                }
                
            # STEP TYPE: giveaway
            if _event.event_type == "giveaway":
                for _statistic in _event.statistics:
                    if _statistic.type == 'giveaway':
                            player = _statistic.player.full_name

                eventReportObj = {
                    'type': _event.event_type,
                    'action_area' :  action_area ,
                    'location' : [_event.location.coord_x, _event.location.coord_y ]
                }
                
            # STEP TYPE: penalty
            if _event.event_type == "penalty":
                for _statistic in _event.statistics:
                    if _statistic.type == 'penalty':
                            if hasattr(_statistic,'player'):
                                player = _statistic.player.full_name

                if player:
                    eventReportObj = {
                        'penalty_type': _event.penalty_type,
                        'penalty_duration': _event.duration,
                        'type': _event.event_type,
                        'action_area' :  action_area ,
                        'location' : [_event.location.coord_x, _event.location.coord_y ]
                    }
            
            
            # STEP TYPE: faceoff
            if _event.event_type == "faceoff":
                _eventName = ''
                for _statistic in _event.statistics:
                    if _statistic.type == 'faceoff':
                            player = _statistic.player.full_name
                            _eventName = 'faceoff'
                            
                            if hasattr(_statistic,'win'):
                                _eventName += '_win'
                            else:
                                _eventName += '_lost'
    
                eventReportObj = {
                    'type': _eventName,
                    'action_area' :  action_area ,
                    'location' : [_event.location.coord_x, _event.location.coord_y ]
                }
               
            # STEP: add item report
            if eventReportObj:
                if player not in report[team]:
                    report[team][player] = []

                report[team][player].append(eventReportObj)
        

# Debug

In [211]:
gameFileId = '506c3ccd-61c1-4170-b19f-751e6ccae992' # oilers_games[0]

with open( f"./output/games/game_{gameFileId}.json") as f:
    data = json.load(f)

_gameObj = DictObj(data)

In [212]:
# !!no print!!
#parseGame(_gameObj)
#print(report)

{'Panthers': {'Sam Bennett': [{'type': 'faceoff_win', 'clock': '20:00', 'action_area': 'neutralzone', 'location': [1188, 504]}, {'type': 'giveaway', 'clock': '2:00', 'action_area': 'defensivezone', 'location': [2028, 948]}, {'type': 'hit', 'action_area': 'defensivezone', 'location': [204, 72]}, {'type': 'giveaway', 'clock': '3:53', 'action_area': 'defensivezone', 'location': [132, 168]}, {'type': 'hit', 'action_area': 'defensivezone', 'location': [276, 36]}, {'type': 'faceoff_win', 'clock': '15:46', 'action_area': 'insideleftcircle', 'location': [360, 768]}, {'type': 'hit', 'action_area': 'outsiderightcircle', 'location': [348, 24]}, {'penalty_type': 'Cross Checking', 'penalty_duration': 2, 'type': 'penalty', 'clock': '11:03', 'action_area': 'lowslot', 'location': [192, 492]}, {'type': 'hit', 'action_area': 'defensivezone', 'location': [2340, 216]}, {'type': 'hit', 'action_area': 'downlow', 'location': [108, 360]}, {'type': 'hit', 'action_area': 'defensivezone', 'location': [1908, 684]

# Loop all games

In [5]:

for gameFileId in final_games:
    with open( f"./output/games/game_{gameFileId}.json") as f:
        data = json.load(f)
    _gameObj = DictObj(data)
    try:
        parseGame(_gameObj)
    except:
        print(gameFileId)

In [6]:
file_name = f"./output/reports/widget-final-events/widget-final-events.json"
with open(file_name, "w") as outfile:
    json.dump(report, outfile, indent=2)